In [1]:
import pandas as pd
import iris
import glob
import sys
import os
import time
import warnings
import pickle
from collections import OrderedDict

from Identify_Events_Functions import *
from Prepare_Data_Functions import *

pd.set_option('display.float_format', '{:.3f}'.format)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning, message=".*'+init=<authority>:<code>' syntax is deprecated.*")

######################################################
### Define which rainfall data we are looking for events in
######################################################
dataset_name = 'filtered_100'
dataset_path_pattern = '/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/*'

######################################################
### Define data for finding the indepedent events at each gauge
######################################################

# Get Tb0 values at each gauge
tbo_vals = pd.read_csv('/nfs/a319/gy17m2a/PhD/datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# Read in a sample cube for finding the location of gauge in grid
yr=2006
sample_cube = iris.load(f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{yr}/metoffice-c-band-rain-radar_uk_{yr}0602.nc')[0][1,:,:]

######################################################
### Get all the 5 minute data for one year, into one cube
# (if it already exists in a pickle file, then load it from there)
######################################################
general_filename = dataset_path_pattern.format(year=yr)
pickle_file_filepath = f"/nfs/a319/gy17m2a/PhD/datadir/cache/nimrod_5mins/unfiltered/WholeYear/cube_{yr}.pkl"

if os.path.exists(pickle_file_filepath):
    print("Pickle file exists, so loading that")
    full_year_cube = load_cube_from_picklefile(pickle_file_filepath)
else:
    print("Pickle file doesnt exist, so creating and then saving that")
    
    ### Get the data filepaths
    print(f"Loading data for year {yr}")
    
    # Create cube list
    cubes = load_files_to_cubelist(yr, general_filename)
    
    # Clean cubes of things which are problematic for concatenation
    cubes = clean_cubes(cubes)
    
    # Join them into one (with error handling to deal with times which are wrong)
    try:
        full_year_cube = cubes.concatenate_cube()
        print("Concatenation successful!")
    except Exception as e:
        print(f"Initial concatenation failed: {str(e)}")

        # If initial concatenation fails, remove problematic cubes and try again
        try:
            full_year_cube = remove_problematic_cubes(cubes)
            print("Concatenation successful after removing problematic cubes!")
        except RuntimeError as e:
            print(f"Concatenation failed after removing problematic cubes: {str(e)}")               
    save_cube_as_pickle_file(full_year_cube, pickle_file_filepath)

######################################################
# Find events at each gauge
######################################################
failed_gauges = []
for yr in range(2015,2020):
    print(yr)
    gauge_nums = range(300,320)
    # Function to process each gauge
    for gauge_num in gauge_nums:
        if not gauge_num in [423, 444, 827, 888]:
                print(f"gauge num is {gauge_num}")

                ######################################################
                ## Check if any files are missing, across the 3 filtering options
                # If there are: code will continue to run
                # If not: code will move to next gauge
                ######################################################
                for dataset_name in ['unfiltered', 'filtered_100', 'filtered_300']:
                    # Create a flag to record whether we are missing any of the files we need
                    missing_files = False
                    # Define directory filepath which will store results
                    base_dir = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_5mins/NIMROD_1km_{dataset_name}/{gauge_num}/WholeYear"
                    # Create the directory if it doesnt exist
                    if not os.path.isdir(base_dir):
                        os.makedirs(base_dir)
                    # Check if we are missing any of the files, and if so, change the flag to True
                    if not any(os.path.exists(f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv") for duration in [0.5, 1, 2, 3, 6, 12, 24]):
                        missing_files = True

                # If we are missing some files then get the data for the grid cell, 
                if missing_files:
                    # Find the Tb0 and index of this gauge
                    Tb0, idx_2d = find_gauge_Tb0_and_location_in_grid(tbo_vals, gauge_num, sample_cube)

                    # Extract data for the specified indices
                    start= time.time()
                    one_location_cube = full_year_cube[:, idx_2d[0], idx_2d[1]]
                    data = one_location_cube.data
                    end=time.time()
                    print(f"Time to load data is {round(end-start,2)} seconds")

                    ##### Filter cube according to different options
                    # Find events with filtered cubes
                    filtering_dict = {1000000:'unfiltered', 300:'filtered_300',100:'filtered_100'}
                    for filtering_key, dataset_name in filtering_dict.items():
                        print(f"running for {dataset_name}")
                        # Create cube with filterings applied
                        cube = filtered_cube(one_location_cube,  filter_above=filtering_key)
                        print("reloading data")
                        data = cube.data
                        print(f"max value is {np.nanmax(cube.data)}")
                        # Convert to dataframe
                        df = create_df_with_gaps_filled_in(cube, data, time_resolution = 5)
                        # Search dataframe for events corresponding to durations
                        for duration in [0.5, 1, 2, 3, 6, 12, 24]:
                            base_dir = f"/nfs/a161/gy17m2a/PhD/ProcessedData/IndependentEvents/NIMROD_5mins/NIMROD_1km_{dataset_name}/{gauge_num}/WholeYear"

                            filename =  f"{base_dir}/{duration}hrs_{yr}_v2_part0.csv"
                            if not os.path.exists(filename):
                                print(f"Finding the AMAX for {duration}hr events for gauge {gauge_num} in year {yr} for {dataset_name}")
                                # Find events
                                events_v2 = search_for_valid_events(df, duration=duration, Tb0=Tb0)

                                # Save events to CSV
                                for num, event in enumerate(events_v2):
                                    if len(event) > 1:
                                            event.to_csv(f"{base_dir}/{duration}hrs_{yr}_v2_part{num}.csv")
                                            if event['precipitation (mm/hr)'].isna().any():
                                                print("NANs in this event")
                            else:
                                print(f"already exists{filename}")
                                pass   

    print(f"failed gauges are: {failed_gauges}")


Pickle file exists, so loading that
2015
gauge num is 300
gauge num is 301
gauge num is 302
gauge num is 303
gauge num is 304
gauge num is 305
gauge num is 306
gauge num is 307
gauge num is 308
gauge num is 309
gauge num is 310
gauge num is 311
gauge num is 312
gauge num is 313
gauge num is 314
gauge num is 315
gauge num is 316
gauge num is 317
gauge num is 318
gauge num is 319
failed gauges are: []
2016
gauge num is 300
gauge num is 301
gauge num is 302
gauge num is 303
gauge num is 304
gauge num is 305
gauge num is 306
gauge num is 307
gauge num is 308
gauge num is 309
gauge num is 310
gauge num is 311
gauge num is 312
gauge num is 313
gauge num is 314
gauge num is 315
gauge num is 316
gauge num is 317
gauge num is 318
gauge num is 319
failed gauges are: []
2017
gauge num is 300
gauge num is 301
gauge num is 302
gauge num is 303
gauge num is 304
gauge num is 305
gauge num is 306
gauge num is 307
gauge num is 308
gauge num is 309
gauge num is 310
gauge num is 311
gauge num is 312
gaug

Finding the AMAX for 12hr events for gauge 317 in year 2017 for unfiltered
Event doesnt contain NAN, total event precip is 22.127605438232422
Finding the AMAX for 24hr events for gauge 317 in year 2017 for unfiltered
Event doesnt contain NAN, total event precip is 22.127605438232422
running for filtered_300
reloading data
max value is 50.0
Finding the AMAX for 0.5hr events for gauge 317 in year 2017 for filtered_300
Event contains NAN, total event precip is 27.591144561767578
Event doesnt contain NAN, total event precip is 15.955730438232422
Finding the AMAX for 1hr events for gauge 317 in year 2017 for filtered_300
Event doesnt contain NAN, total event precip is 15.955730438232422
Finding the AMAX for 2hr events for gauge 317 in year 2017 for filtered_300
Event doesnt contain NAN, total event precip is 16.460939407348633
Finding the AMAX for 3hr events for gauge 317 in year 2017 for filtered_300
Event doesnt contain NAN, total event precip is 15.955730438232422
Finding the AMAX for 6h

PermissionError: [Errno 13] Permission denied: b'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2006/metoffice-c-band-rain-radar_uk_20060602.nc'